In [1]:
import numpy as np
import keras
import re
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


### Read Data

In [2]:
with open('embeddings-twitter.txt',encoding='utf-8') as f:
    emb_raw= f.read().splitlines()

In [3]:
with open('tweet-pos/tweets-train.txt',encoding='utf-8') as f:
    train = list(map(lambda x: re.split(r'\n|\t', x), f.read().split("\n\n")[:-1]))

with open('tweet-pos/tweets-dev.txt',encoding='utf-8') as f:
    dev = list(map(lambda x: re.split(r'\n|\t', x), f.read().split("\n\n")[:-1]))

with open('tweet-pos/tweets-devtest.txt',encoding='utf-8') as f:
    devtest = list(map(lambda x: re.split(r'\n|\t', x), f.read().split("\n\n")[:-1]))

### Preprocessing

In [4]:
emb = {}
for entry in emb_raw:
    l = entry.split()
    emb[l[0]] = np.array(list(map(float, l[1:])))

In [5]:
unk = emb["UUUNKKK"]

In [6]:
def get_emb(emb, word):
    return emb.get(word, unk)

In [7]:
def preprocessing(dset, w, emb):
    inputs = []
    labels = []
    for s in dset:
        tokens = s[::2]
        n = len(tokens)
        labels += s[1::2]
        embeddings = [get_emb(emb, '<s>')]*w +\
                     [get_emb(emb, token) for token in tokens] +\
                     [get_emb(emb, '</s>')]*w
        for i in range(w, w+n):
            inputs.append(np.concatenate(embeddings[i-w:i+w+1]))
    x = np.array(inputs)
    categories = set(labels)
    return x, labels, categories

In [8]:
train_x, train_labels, train_categories = preprocessing(train, 1, emb)
dev_x, dev_labels, dev_categories = preprocessing(dev, 1, emb)
devtest_x, devtest_labels, devtest_categories = preprocessing(devtest, 1, emb)

In [9]:
n = len(train_categories)

In [10]:
train_categories - dev_categories

{'M', 'Y'}

In [11]:
train_categories - devtest_categories

{'M', 'Y'}

In [12]:
encoder = LabelEncoder()
encoder.fit(train_labels)
train_y = to_categorical(encoder.transform(train_labels))
dev_y = to_categorical(encoder.transform(dev_labels))
devtest_y = to_categorical(encoder.transform(devtest_labels))

### Baseline Tagger (w=1)

In [13]:
callbacks = [EarlyStopping(monitor='val_acc', verbose=1)]

In [14]:
model = Sequential()
model.add(Dense(128, activation='tanh', input_dim=150))
model.add(Dense(n, activation='softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=50, callbacks=callbacks, validation_data=(dev_x, dev_y))

Train on 14619 samples, validate on 4823 samples
Epoch 1/50
14619/14619 [==============================] - 1s 78us/step - loss: 2.6491 - acc: 0.3329 - val_loss: 2.2129 - val_acc: 0.4564
Epoch 2/50
14619/14619 [==============================] - 1s 67us/step - loss: 1.9506 - acc: 0.5355 - val_loss: 1.7563 - val_acc: 0.6050
Epoch 3/50
14619/14619 [==============================] - 1s 65us/step - loss: 1.5850 - acc: 0.6343 - val_loss: 1.4785 - val_acc: 0.6469
Epoch 4/50
14619/14619 [==============================] - 1s 66us/step - loss: 1.3528 - acc: 0.6734 - val_loss: 1.2952 - val_acc: 0.6795
Epoch 5/50
14619/14619 [==============================] - 1s 66us/step - loss: 1.1950 - acc: 0.7057 - val_loss: 1.1657 - val_acc: 0.7074
Epoch 6/50
14619/14619 [==============================] - 1s 66us/step - loss: 1.0814 - acc: 0.7336 - val_loss: 1.0706 - val_acc: 0.7276
Epoch 7/50
14619/14619 [==============================] - 1s 66us/step - loss: 0.9958 - acc: 0.7489 - val_loss: 0.9967 - val_acc:

In [15]:
test_loss, test_acc = model.evaluate(devtest_x, devtest_y)
print('test_acc:', test_acc)

7152/7152 [==============================] - 0s 28us/step
test_acc: 0.8358501118568232


### Vary w

#### w=0 

In [16]:
train_x_w0 = preprocessing(train, 0, emb)[0]
dev_x_w0 = preprocessing(dev, 0, emb)[0]
devtest_x_w0 = preprocessing(devtest, 0, emb)[0]

In [17]:
model_w0 = Sequential()
model_w0.add(Dense(128, activation='tanh', input_dim=50))
model_w0.add(Dense(n, activation='softmax'))
model_w0.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model_w0.fit(train_x_w0, train_y, epochs=50, callbacks=callbacks, validation_data=(dev_x_w0, dev_y))

Train on 14619 samples, validate on 4823 samples
Epoch 1/50
14619/14619 [==============================] - 1s 87us/step - loss: 2.7477 - acc: 0.3091 - val_loss: 2.3430 - val_acc: 0.4329
Epoch 2/50
14619/14619 [==============================] - 1s 74us/step - loss: 2.0795 - acc: 0.5260 - val_loss: 1.8709 - val_acc: 0.6069
Epoch 3/50
14619/14619 [==============================] - 1s 62us/step - loss: 1.6953 - acc: 0.6211 - val_loss: 1.5669 - val_acc: 0.6513
Epoch 4/50
14619/14619 [==============================] - 1s 55us/step - loss: 1.4415 - acc: 0.6667 - val_loss: 1.3657 - val_acc: 0.6836
Epoch 5/50
14619/14619 [==============================] - 1s 65us/step - loss: 1.2710 - acc: 0.7013 - val_loss: 1.2271 - val_acc: 0.7068
Epoch 6/50
14619/14619 [==============================] - 1s 59us/step - loss: 1.1524 - acc: 0.7216 - val_loss: 1.1285 - val_acc: 0.7220
Epoch 7/50
14619/14619 [==============================] - 1s 52us/step - loss: 1.0662 - acc: 0.7406 - val_loss: 1.0550 - val_acc:

In [18]:
test_loss, test_acc = model_w0.evaluate(devtest_x_w0, devtest_y)
print('test_acc:', test_acc)

7152/7152 [==============================] - 0s 25us/step
test_acc: 0.8069071588366891


#### w=2 

In [19]:
train_x_w2 = preprocessing(train, 2, emb)[0]
dev_x_w2 = preprocessing(dev, 2, emb)[0]
devtest_x_w2 = preprocessing(devtest, 2, emb)[0]

In [20]:
model_w2 = Sequential()
model_w2.add(Dense(128, activation='tanh', input_dim=250))
model_w2.add(Dense(n, activation='softmax'))
model_w2.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model_w2.fit(train_x_w2, train_y, epochs=50, callbacks=callbacks, validation_data=(dev_x_w2, dev_y))

Train on 14619 samples, validate on 4823 samples
Epoch 1/50
14619/14619 [==============================] - 1s 72us/step - loss: 2.6682 - acc: 0.2887 - val_loss: 2.2438 - val_acc: 0.4201
Epoch 2/50
14619/14619 [==============================] - 1s 60us/step - loss: 1.9860 - acc: 0.5090 - val_loss: 1.8057 - val_acc: 0.5810
Epoch 3/50
14619/14619 [==============================] - 1s 60us/step - loss: 1.6289 - acc: 0.6192 - val_loss: 1.5269 - val_acc: 0.6407
Epoch 4/50
14619/14619 [==============================] - 1s 60us/step - loss: 1.3903 - acc: 0.6719 - val_loss: 1.3355 - val_acc: 0.6763
Epoch 5/50
14619/14619 [==============================] - 1s 58us/step - loss: 1.2235 - acc: 0.7074 - val_loss: 1.1980 - val_acc: 0.7062
Epoch 6/50
14619/14619 [==============================] - 1s 59us/step - loss: 1.1023 - acc: 0.7296 - val_loss: 1.0966 - val_acc: 0.7236
Epoch 7/50
14619/14619 [==============================] - 1s 61us/step - loss: 1.0106 - acc: 0.7468 - val_loss: 1.0181 - val_acc:

In [21]:
test_loss, test_acc = model_w2.evaluate(devtest_x_w2, devtest_y)
print('test_acc:', test_acc)

7152/7152 [==============================] - 0s 33us/step
test_acc: 0.8323545861297539


### Change non-linearity functions

#### identity

In [22]:
model_i = Sequential()
model_i.add(Dense(128, activation='linear', input_dim=150))
model_i.add(Dense(n, activation='softmax'))
model_i.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model_i.fit(train_x, train_y, epochs=50, callbacks=callbacks, validation_data=(dev_x, dev_y))

Train on 14619 samples, validate on 4823 samples
Epoch 1/50
14619/14619 [==============================] - 1s 67us/step - loss: 2.6353 - acc: 0.3212 - val_loss: 2.1784 - val_acc: 0.4653
Epoch 2/50
14619/14619 [==============================] - 1s 55us/step - loss: 1.8971 - acc: 0.5481 - val_loss: 1.7064 - val_acc: 0.6015
Epoch 3/50
14619/14619 [==============================] - 1s 54us/step - loss: 1.5244 - acc: 0.6374 - val_loss: 1.4280 - val_acc: 0.6591
Epoch 4/50
14619/14619 [==============================] - 1s 56us/step - loss: 1.2934 - acc: 0.6909 - val_loss: 1.2476 - val_acc: 0.7000
Epoch 5/50
14619/14619 [==============================] - 1s 54us/step - loss: 1.1389 - acc: 0.7246 - val_loss: 1.1210 - val_acc: 0.7211
Epoch 6/50
14619/14619 [==============================] - 1s 55us/step - loss: 1.0285 - acc: 0.7459 - val_loss: 1.0280 - val_acc: 0.7423
Epoch 7/50
14619/14619 [==============================] - 1s 54us/step - loss: 0.9455 - acc: 0.7630 - val_loss: 0.9562 - val_acc:

In [23]:
test_loss, test_acc = model_i.evaluate(devtest_x, devtest_y)
print('test_acc:', test_acc)

7152/7152 [==============================] - 0s 33us/step
test_acc: 0.8386465324384788


#### ReLU 

In [24]:
model_relu = Sequential()
model_relu.add(Dense(128, activation='relu', input_dim=150))
model_relu.add(Dense(n, activation='softmax'))
model_relu.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model_relu.fit(train_x, train_y, epochs=50, callbacks=callbacks, validation_data=(dev_x, dev_y))

Train on 14619 samples, validate on 4823 samples
Epoch 1/50
14619/14619 [==============================] - 1s 94us/step - loss: 2.8691 - acc: 0.2014 - val_loss: 2.5484 - val_acc: 0.2874
Epoch 2/50
14619/14619 [==============================] - 1s 95us/step - loss: 2.2951 - acc: 0.3977 - val_loss: 2.0975 - val_acc: 0.4748
Epoch 3/50
14619/14619 [==============================] - 1s 92us/step - loss: 1.8942 - acc: 0.5449 - val_loss: 1.7586 - val_acc: 0.5922
Epoch 4/50
14619/14619 [==============================] - 1s 90us/step - loss: 1.5972 - acc: 0.6289 - val_loss: 1.5124 - val_acc: 0.6428
Epoch 5/50
14619/14619 [==============================] - 1s 68us/step - loss: 1.3829 - acc: 0.6700 - val_loss: 1.3338 - val_acc: 0.6768
Epoch 6/50
14619/14619 [==============================] - 1s 89us/step - loss: 1.2280 - acc: 0.6994 - val_loss: 1.2028 - val_acc: 0.7070
Epoch 7/50
14619/14619 [==============================] - 1s 85us/step - loss: 1.1128 - acc: 0.7247 - val_loss: 1.1029 - val_acc:

In [25]:
test_loss, test_acc = model_relu.evaluate(devtest_x, devtest_y)
print('test_acc:', test_acc)

7152/7152 [==============================] - 1s 73us/step
test_acc: 0.8358501118568232


#### Sigmoid 

In [26]:
model_sig = Sequential()
model_sig.add(Dense(128, activation='sigmoid', input_dim=150))
model_sig.add(Dense(n, activation='softmax'))
model_sig.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model_sig.fit(train_x, train_y, epochs=50, callbacks=callbacks, validation_data=(dev_x, dev_y))

Train on 14619 samples, validate on 4823 samples
Epoch 1/50
14619/14619 [==============================] - 2s 161us/step - loss: 2.7327 - acc: 0.1696 - val_loss: 2.6573 - val_acc: 0.1951
Epoch 2/50
14619/14619 [==============================] - 2s 128us/step - loss: 2.6095 - acc: 0.2045 - val_loss: 2.6017 - val_acc: 0.2710
Epoch 3/50
14619/14619 [==============================] - 2s 121us/step - loss: 2.5511 - acc: 0.2398 - val_loss: 2.5386 - val_acc: 0.2826
Epoch 4/50
14619/14619 [==============================] - 2s 129us/step - loss: 2.4914 - acc: 0.2833 - val_loss: 2.4756 - val_acc: 0.3067
Epoch 5/50
14619/14619 [==============================] - 1s 97us/step - loss: 2.4276 - acc: 0.3320 - val_loss: 2.4121 - val_acc: 0.3471
Epoch 6/50
14619/14619 [==============================] - 1s 77us/step - loss: 2.3594 - acc: 0.3785 - val_loss: 2.3408 - val_acc: 0.3962
Epoch 7/50
14619/14619 [==============================] - 1s 74us/step - loss: 2.2878 - acc: 0.4166 - val_loss: 2.2696 - val_

In [27]:
test_loss, test_acc = model_sig.evaluate(devtest_x, devtest_y)
print('test_acc:', test_acc)

7152/7152 [==============================] - 1s 70us/step
test_acc: 0.7455257270693513


### Change hidden layers 

#### No hidden layer 

In [28]:
model_l0 = Sequential()
model_l0.add(Dense(n, activation='softmax', input_dim=150))
model_l0.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model_l0.fit(train_x, train_y, epochs=50, callbacks=callbacks, validation_data=(dev_x, dev_y))

Train on 14619 samples, validate on 4823 samples
Epoch 1/50
14619/14619 [==============================] - 2s 138us/step - loss: 2.9527 - acc: 0.1958 - val_loss: 2.7045 - val_acc: 0.3079
Epoch 2/50
14619/14619 [==============================] - 1s 92us/step - loss: 2.5120 - acc: 0.3523 - val_loss: 2.3638 - val_acc: 0.4027
Epoch 3/50
14619/14619 [==============================] - 1s 83us/step - loss: 2.2267 - acc: 0.4543 - val_loss: 2.1319 - val_acc: 0.4935
Epoch 4/50
14619/14619 [==============================] - 1s 90us/step - loss: 2.0229 - acc: 0.5281 - val_loss: 1.9600 - val_acc: 0.5501
Epoch 5/50
14619/14619 [==============================] - 1s 89us/step - loss: 1.8674 - acc: 0.5764 - val_loss: 1.8257 - val_acc: 0.5886
Epoch 6/50
14619/14619 [==============================] - 1s 95us/step - loss: 1.7439 - acc: 0.6104 - val_loss: 1.7177 - val_acc: 0.6148
Epoch 7/50
14619/14619 [==============================] - 1s 80us/step - loss: 1.6433 - acc: 0.6364 - val_loss: 1.6288 - val_acc

In [29]:
test_loss, test_acc = model_l0.evaluate(devtest_x, devtest_y)
print('test_acc:', test_acc)

7152/7152 [==============================] - 0s 27us/step
test_acc: 0.799496644295302


#### 1 Hidden layer with width 256 

In [30]:
model_l1_l = Sequential()
model_l1_l.add(Dense(256, activation='tanh', input_dim=150))
model_l1_l.add(Dense(n, activation='softmax'))
model_l1_l.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model_l1_l.fit(train_x, train_y, epochs=50, callbacks=callbacks, validation_data=(dev_x, dev_y))

Train on 14619 samples, validate on 4823 samples
Epoch 1/50
14619/14619 [==============================] - 1s 93us/step - loss: 2.5429 - acc: 0.3413 - val_loss: 2.0813 - val_acc: 0.4868
Epoch 2/50
14619/14619 [==============================] - 1s 66us/step - loss: 1.8193 - acc: 0.5740 - val_loss: 1.6264 - val_acc: 0.6199
Epoch 3/50
14619/14619 [==============================] - 1s 71us/step - loss: 1.4629 - acc: 0.6648 - val_loss: 1.3639 - val_acc: 0.6795
Epoch 4/50
14619/14619 [==============================] - 1s 62us/step - loss: 1.2448 - acc: 0.7108 - val_loss: 1.1928 - val_acc: 0.7174
Epoch 5/50
14619/14619 [==============================] - 1s 67us/step - loss: 1.0993 - acc: 0.7361 - val_loss: 1.0737 - val_acc: 0.7390
Epoch 6/50
14619/14619 [==============================] - 1s 72us/step - loss: 0.9957 - acc: 0.7548 - val_loss: 0.9859 - val_acc: 0.7516
Epoch 7/50
14619/14619 [==============================] - 1s 62us/step - loss: 0.9183 - acc: 0.7695 - val_loss: 0.9193 - val_acc:

In [31]:
test_loss, test_acc = model_l1_l.evaluate(devtest_x, devtest_y)
print('test_acc:', test_acc)

7152/7152 [==============================] - 0s 54us/step
test_acc: 0.8322147651006712


#### 2 hidden layers with width 256 & 128

In [32]:
model_l2_s = Sequential()
model_l2_s.add(Dense(256, activation='tanh', input_dim=150))
model_l2_s.add(Dense(128, activation='tanh'))
model_l2_s.add(Dense(n, activation='softmax'))
model_l2_s.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model_l2_s.fit(train_x, train_y, epochs=50, callbacks=callbacks, validation_data=(dev_x, dev_y))

Train on 14619 samples, validate on 4823 samples
Epoch 1/50
14619/14619 [==============================] - 2s 159us/step - loss: 2.3478 - acc: 0.4125 - val_loss: 1.8016 - val_acc: 0.5893
Epoch 2/50
14619/14619 [==============================] - 2s 103us/step - loss: 1.4966 - acc: 0.6465 - val_loss: 1.2941 - val_acc: 0.6813
Epoch 3/50
14619/14619 [==============================] - 2s 110us/step - loss: 1.1288 - acc: 0.7241 - val_loss: 1.0486 - val_acc: 0.7367
Epoch 4/50
14619/14619 [==============================] - 2s 113us/step - loss: 0.9406 - acc: 0.7609 - val_loss: 0.9124 - val_acc: 0.7607
Epoch 5/50
14619/14619 [==============================] - 2s 117us/step - loss: 0.8291 - acc: 0.7801 - val_loss: 0.8243 - val_acc: 0.7752
Epoch 6/50
14619/14619 [==============================] - 2s 120us/step - loss: 0.7558 - acc: 0.7916 - val_loss: 0.7655 - val_acc: 0.7864
Epoch 7/50
14619/14619 [==============================] - 2s 114us/step - loss: 0.7035 - acc: 0.8033 - val_loss: 0.7208 - v

In [33]:
test_loss, test_acc = model_l2_s.evaluate(devtest_x, devtest_y)
print('test_acc:', test_acc)

7152/7152 [==============================] - 0s 46us/step
test_acc: 0.8414429530201343


#### 2 hidden layers with width 512 & 256

In [34]:
model_l2_l = Sequential()
model_l2_l.add(Dense(512, activation='tanh', input_dim=150))
model_l2_l.add(Dense(256, activation='tanh'))
model_l2_l.add(Dense(n, activation='softmax'))
model_l2_l.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model_l2_l.fit(train_x, train_y, epochs=50, callbacks=callbacks, validation_data=(dev_x, dev_y))

Train on 14619 samples, validate on 4823 samples
Epoch 1/50
14619/14619 [==============================] - 3s 171us/step - loss: 2.1489 - acc: 0.4913 - val_loss: 1.5862 - val_acc: 0.6376
Epoch 2/50
14619/14619 [==============================] - 2s 149us/step - loss: 1.3053 - acc: 0.6959 - val_loss: 1.1504 - val_acc: 0.7257
Epoch 3/50
14619/14619 [==============================] - 2s 158us/step - loss: 1.0011 - acc: 0.7557 - val_loss: 0.9488 - val_acc: 0.7582
Epoch 4/50
14619/14619 [==============================] - 2s 165us/step - loss: 0.8478 - acc: 0.7835 - val_loss: 0.8350 - val_acc: 0.7767
Epoch 5/50
14619/14619 [==============================] - 2s 161us/step - loss: 0.7553 - acc: 0.7973 - val_loss: 0.7612 - val_acc: 0.7858
Epoch 6/50
14619/14619 [==============================] - 2s 171us/step - loss: 0.6932 - acc: 0.8087 - val_loss: 0.7100 - val_acc: 0.7972
Epoch 7/50
14619/14619 [==============================] - 3s 196us/step - loss: 0.6487 - acc: 0.8193 - val_loss: 0.6723 - v

In [35]:
test_loss, test_acc = model_l2_l.evaluate(devtest_x, devtest_y)
print('test_acc:', test_acc)

7152/7152 [==============================] - 1s 98us/step
test_acc: 0.8450782997762863
